TODO

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1. Import the required libraries

In [57]:
# import required libraries
from azure.ai.ml import MLClient, command
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import Environment, BuildContext, JobResourceConfiguration

## 1.2. Configure workspace details and get a handle to the workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ai.ml` to get a handle to the required Azure Machine Learning workspace. We use the default [default azure authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for this tutorial. Check the [configuration notebook](../../../configuration.ipynb) for more details on how to configure credentials and connect to a workspace.

In [58]:
import os
from pyrit.common import default_values

default_values.load_default_env()

# Enter details of your AML workspace
subscription_id = os.environ.get('AZURE_SUBSCRIPTION_ID')
resource_group = os.environ.get('AZURE_RESOURCE_GROUP')
workspace = os.environ.get('AZURE_ML_WORKSPACE_NAME')

In [59]:
# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

# 2. Configure the Command

In [ ]:
env_docker_context = Environment(
    build=BuildContext(path="src"),
    name="pyrit",
    description="PyRIT environment created from a Docker context.",
)
ml_client.environments.create_or_update(env_docker_context)

In [61]:
job = command(
    code="./../../..",  # local path where the code is stored
    command="cd pyrit/optimizers/experiments && python run.py --model_name ${{inputs.model_name}} --setup ${{inputs.setup}} --n_train_data ${{inputs.n_train_data}} --n_test_data ${{inputs.n_test_data}} --n_steps ${{inputs.n_steps}} --batch_size ${{inputs.batch_size}}",
    inputs={
        "model_name": "llama_2", # "all_models",
        "setup": "multiple",
        "n_train_data": 25,
        "n_test_data": 0,
        "n_steps": 500,
        "batch_size": 512,
    },
    environment=f"{env_docker_context.name}:{env_docker_context.version}",
    environment_variables={"HF_TOKEN": os.environ["HF_TOKEN"]},
    display_name="suffix_generation",
    description="Generate a suffix for attacking LLMs.",
    resources = JobResourceConfiguration(
        instance_type="Standard_ND96amsr_A100_v4",
        instance_count=1,
    )
)

## 3. Run the Command 
Using the `MLClient` created earlier, we will now run this Command in the workspace.

In [62]:
# submit the command
returned_job = ml_client.create_or_update(job)